In [2]:
# %%capture
# !pip install transformers datasets torch;
# !pip install transformers[torch]
# !pip install accelerate -U

In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from sklearn.metrics import f1_score
from transformers import Trainer, TrainingArguments
import torch
from datasets import load_dataset

In [4]:
# squad = load_dataset("sberquad", split="train")
# squad = squad.shuffle(seed=42)
# squad = squad.train_test_split(test_size=0.2)

In [5]:
df = load_dataset('lmqg/qag_ruquad')

In [6]:
df_train = df['train'].flatten()
df_test =  df['test'].flatten()
df_val = df['validation'].flatten()

In [7]:
#проверка 
for sub_df in [df_train, df_test, df_val]:
    for i in range(sub_df.shape[0]):
        sample = df['train'][i]
        context = sample['paragraph']
        for answer in sample['answers']:
            ind = context.find(answer)
            if ind == -1:
                print(i)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")

/home/user/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def process_samples(sample):
    context = sample['paragraph'].strip()
    questions = sample['questions']
    answers = sample['answers']
    start_positions, end_positions = [], []
    input_ids, masks = [], []
    for i in range(len(answers)):
        question = questions[i].strip()
        answer = answers[i].strip()
        start_char = context.find(answer)
        end_char = start_char + len(answer)
        
        # if start_char > 512 or end_char > 512:
        #     count += 1
        
        tokenized_data = tokenizer(context, question, max_length=512, truncation=True, padding="max_length")
        
        start_token = tokenized_data.char_to_token(start_char)
        end_token = tokenized_data.char_to_token(end_char - 1)
        if start_token != None and end_token != None:
            start_positions.append(start_token)
            end_positions.append(end_token)
            input_ids.append(tokenized_data['input_ids'])
            masks.append(tokenized_data['attention_mask'])
        
    # return count / len(answers)

    return input_ids, masks, start_positions, end_positions

In [10]:
# prob = 0
# size = df_train.shape[0]
# for i in range(size):
#     prob += process_samples(df_train[i])
# print(100 * prob / size)

In [11]:
from tqdm import tqdm
from datasets import Dataset

In [12]:
all_input_ids, all_masks, all_start_positions, all_end_positions = [], [], [], []
for i, sample in enumerate(tqdm(df_train)):
    input_ids, masks, start_positions, end_positions = process_samples(sample)
    all_input_ids.extend(input_ids)
    all_masks.extend(masks)
    all_start_positions.extend(start_positions)
    all_end_positions.extend(end_positions)

df_train_dict = {'input_ids': all_input_ids,
          'attention_mask': all_masks,
          'start_positions': all_start_positions,
          'end_positions': all_end_positions}
train_df = Dataset.from_dict(df_train_dict)

100%|██████████| 10407/10407 [00:54<00:00, 190.38it/s]


In [13]:
all_input_ids, all_masks, all_start_positions, all_end_positions = [], [], [], []
for i, sample in enumerate(tqdm(df_test)):
    input_ids, masks, start_positions, end_positions = process_samples(sample)
    all_input_ids.extend(input_ids)
    all_masks.extend(masks)
    all_start_positions.extend(start_positions)
    all_end_positions.extend(end_positions)

df_test_dict = {'input_ids': all_input_ids,
          'attention_mask': all_masks,
          'start_positions': all_start_positions,
          'end_positions': all_end_positions}
test_df = Dataset.from_dict(df_test_dict)

100%|██████████| 4017/4017 [00:07<00:00, 550.70it/s]


In [14]:
all_input_ids, all_masks, all_start_positions, all_end_positions = [], [], [], []
for i, sample in enumerate(tqdm(df_val)):
    input_ids, masks, start_positions, end_positions = process_samples(sample)
    all_input_ids.extend(input_ids)
    all_masks.extend(masks)
    all_start_positions.extend(start_positions)
    all_end_positions.extend(end_positions)

df_val_dict = {'input_ids': all_input_ids,
          'attention_mask': all_masks,
          'start_positions': all_start_positions,
          'end_positions': all_end_positions}
val_df = Dataset.from_dict(df_val_dict)

100%|██████████| 4079/4079 [00:07<00:00, 549.56it/s]


In [17]:
import pandas as pd
pd.DataFrame(train_df).to_csv('train.csv', index=True)
pd.DataFrame(test_df).to_csv('test.csv', index=True)
pd.DataFrame(val_df).to_csv('val.csv', index=True)

In [18]:
df = load_dataset('csv', data_files={'train': 'train.csv',
                                     'test': 'test.csv',
                                     'validation': 'val.csv'})
df

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 32750
    })
    test: Dataset({
        features: ['Unnamed: 0', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 4084
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 4103
    })
})

In [20]:
df.push_to_hub('Eka-Korn/qa_russian')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Eka-Korn/qa_russian/commit/4b0092f8017c87357d4a58a0a7ee6c0e01f81c1e', commit_message='Upload dataset', commit_description='', oid='4b0092f8017c87357d4a58a0a7ee6c0e01f81c1e', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

In [22]:
columns_to_return = ['input_ids','attention_mask', 'start_positions', 'end_positions']
train_df.set_format(type='pt', columns=columns_to_return)
test_df.set_format(type='pt', columns=columns_to_return)
val_df.set_format(type='pt', columns=columns_to_return)

In [26]:
def compute_f1_metrics(pred):
    start_labels = pred.label_ids[0]
    start_preds = pred.predictions[0].argmax(-1)
    end_labels = pred.label_ids[1]
    end_preds = pred.predictions[1].argmax(-1)

    f1_start = f1_score(start_labels, start_preds, average='macro')
    f1_end = f1_score(end_labels, end_preds, average='macro')

    return {
        'f1_start': f1_start,
        'f1_end': f1_end,
    }

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [25]:
# torch.cuda.set_device(0)
# torch.cuda.current_device()

In [27]:
# Training the model may take around 15 minutes.

training_args = TrainingArguments(
    output_dir='model_results5',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,            # directory for storing logs
    logging_steps=50
)

trainer = Trainer(
    model=model, # the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_df, # training dataset
    eval_dataset=val_df, # evaluation dataset
    compute_metrics=compute_f1_metrics
)

trainer.train()

/home/user/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


[2024-09-28 14:35:35,779] [WARNING] [real_accelerator.py:162:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2024-09-28 14:35:35,782] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cpu (auto detect)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# The evaluation may take around 30 seconds
trainer.evaluate(test_df)

##Use the Fine_Tuned model for inference
We would use our model for inference on the context from Question_Answering_Project and see if we would get a better result

In [19]:
text = r"""
The Golden Age of Comic Books describes an era of American comic books from the
late 1930s to circa 1950. During this time, modern comic books were first published
and rapidly increased in popularity. The superhero archetype was created and many
well-known characters were introduced, including Superman, Batman, Captain Marvel
(later known as SHAZAM!), Captain America, and Wonder Woman.
Between 1939 and 1941 Detective Comics and its sister company, All-American Publications,
introduced popular superheroes such as Batman and Robin, Wonder Woman, the Flash,
Green Lantern, Doctor Fate, the Atom, Hawkman, Green Arrow and Aquaman.[7] Timely Comics,
the 1940s predecessor of Marvel Comics, had million-selling titles featuring the Human Torch,
the Sub-Mariner, and Captain America.[8]
As comic books grew in popularity, publishers began launching titles that expanded
into a variety of genres. Dell Comics' non-superhero characters (particularly the
licensed Walt Disney animated-character comics) outsold the superhero comics of the day.[12]
The publisher featured licensed movie and literary characters such as Mickey Mouse, Donald Duck,
Roy Rogers and Tarzan.[13] It was during this era that noted Donald Duck writer-artist
Carl Barks rose to prominence.[14] Additionally, MLJ's introduction of Archie Andrews
in Pep Comics #22 (December 1941) gave rise to teen humor comics,[15] with the Archie
Andrews character remaining in print well into the 21st century.[16]
At the same time in Canada, American comic books were prohibited importation under
the War Exchange Conservation Act[17] which restricted the importation of non-essential
goods. As a result, a domestic publishing industry flourished during the duration
of the war which were collectively informally called the Canadian Whites.
The educational comic book Dagwood Splits the Atom used characters from the comic
strip Blondie.[18] According to historian Michael A. Amundson, appealing comic-book
characters helped ease young readers' fear of nuclear war and neutralize anxiety
about the questions posed by atomic power.[19] It was during this period that long-running
humor comics debuted, including EC's Mad and Carl Barks' Uncle Scrooge in Dell's Four
Color Comics (both in 1952).[20][21]
"""

questions = ["What superheroes were introduced between 1939 and 1941 by Detective Comics and its sister company?",
             "What comic book characters were created between 1939 and 1941?",
             "What well-known characters were created between 1939 and 1941?",
             "What well-known superheroes were introduced between 1939 and 1941 by Detective Comics?"]

In [20]:
for question in questions:
    inputs = tokenizer.encode_plus(question, text, return_tensors="pt")
    #print("inputs", inputs)
    #print("inputs", type(inputs))
    input_ids = inputs["input_ids"].tolist()[0]
    inputs.to("cuda")

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_model = model(**inputs)

    answer_start = torch.argmax(
        answer_model['start_logits']
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_model['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What superheroes were introduced between 1939 and 1941 by Detective Comics and its sister company?
Answer: Superman, Batman, Captain Marvel ( later known as SHAZAM! ), Captain America, and Wonder Woman

Question: What comic book characters were created between 1939 and 1941?
Answer: Superman, Batman, Captain Marvel ( later known as SHAZAM! ), Captain America, and Wonder Woman

Question: What well-known characters were created between 1939 and 1941?
Answer: Superman, Batman, Captain Marvel ( later known as SHAZAM! ), Captain America, and Wonder Woman

Question: What well-known superheroes were introduced between 1939 and 1941 by Detective Comics?
Answer: Superman, Batman, Captain Marvel ( later known as SHAZAM! ), Captain America, and Wonder Woman



##Conclusion
We can improve performance of a pre - trained model on our specific set of data through fine - tuning, Hugging Face is a platfrom which helps with this, as most of our pipeline method, pre - trained model & fine - tuning method comes from Hugging Face, we enables us to get better performance on our specific dataset.